In [126]:
import pandas as pd

from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

report_date - дата среза фичей

есть есть еденица мы не знаем какая она 1 и 2 или 3е

In [8]:
raw_data = pd.read_csv('data/train_dataset_Самолет.csv', low_memory=False)

In [174]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14456 entries, 0 to 14455
Columns: 2666 entries, report_date to col2663
dtypes: float64(2316), int64(3), object(347)
memory usage: 294.0+ MB


In [64]:
raw_data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


In [105]:
raw_data.iloc[1,3:-1].dropna().unique()

array([1.0, 0.0, 4.0, 19.2, 19.200000762939453, 92.55, 92.5500030517578,
       30.0, 5407132.0, 23789540.0, 715.0,
       'https://samolet.ru/flats/?project=7', 'filter_price_asc', 55.7483,
       55.74829864501953, 37.6171, 37.61709976196289, 'page_load',
       'Mobile Safari', 'iPhone', 'iOS', '/project/sputnik/', 'sms',
       'exponea', 'Russia', 'Moscow',
       '01febac0-b083-494e-8589-f98400074b94', 6.0, 2.0, 8193703.78,
       5000.0, 1420568.84333333, 277709.27, 92569.75666667, 3.0, 3.5,
       204857.0, 95379.0, 150118.0, 7938208.75, 318565.86, 4128387.305,
       38.75, 3.34, 21.045, 242.0, 5903704.0, 362.0, 7616803.0],
      dtype=object)

In [199]:
# Какие-то ссылки
raw_data.iloc[:,54].dropna().unique()

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
14451    NaN
14452    NaN
14453    NaN
14454    NaN
14455    NaN
Name: col52, Length: 14456, dtype: object

In [36]:
num_unq_clients = raw_data['client_id'].unique().shape[0] # Number of unque clients
num_unq_clients

4817

## Full model

In [276]:
target = raw_data.iloc[:,2]
data = raw_data.drop(columns=['target', 'report_date'])
data

,client_id,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,1241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,1969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,7116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,7117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


In [277]:
# Колонки с категориальными данными
catCols = [col for col in data.columns if data[col].dtype=="object"]
print(f'number cat columns = {len(catCols)}')

number cat columns = 346


In [278]:
categorical_columns = data[catCols]
numeric_columns = data.drop(catCols, axis=1)

In [279]:
# TT split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=4)

In [283]:
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
import numpy as np

from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        make_column_selector(dtype_include="object"),
    ),
    remainder="passthrough",
)

hist_one_hot = make_pipeline(
    one_hot_encoder, HistGradientBoostingClassifier(random_state=4)
)

In [284]:
from sklearn.model_selection import cross_validate
scoring = "neg_mean_absolute_percentage_error"
one_hot_result = cross_validate(hist_one_hot, X_train, y_train, cv=3, scoring=scoring)

In [223]:
columns_with_date = data.columns[data.isin(['main']).any()]
columns_with_date

Index(['col57', 'col58', 'col59', 'col60', 'col61', 'col62', 'col63', 'col64',
       'col129', 'col130', 'col131', 'col132', 'col133', 'col134', 'col135',
       'col136', 'col1046', 'col1047', 'col1048', 'col1049', 'col1050',
       'col1051', 'col1052', 'col1054', 'col1055', 'col1056', 'col1057',
       'col1058', 'col1059', 'col1060'],
      dtype='object')